# 初始化

In [ ]:
# 移动目录
# %cd /data/homework/user/wangning/aitutor/knowledge_aigc_k12_aivideo_unmanned_server
# %cd /data/homework/user/wangning/aitutor/xg

import pandas as pd
import numpy as np
import json
import csv
import requests
import sys
import math
import re

%pwd

# 常用工具型函数

In [ ]:
# sft数据生成
def gen_share_gpt_format_data(in_str, out_str):
    data = {
        "conversations": [
            {
                "from": "human",
                "value": in_str 
            },
            {
                "from": "gpt",
                "value": out_str
            }
        ]
    }
    return data


# dpo 格式数据生成
def gen_dpo_format_data(in_str, good_str, bad_str):
    data = {
        "conversations": [
            {"from": "human", "value": in_str }
        ],
        "chosen": {"from": "gpt", "value": good_str},
        "rejected": {"from": "gpt","value": bad_str}
    }
    return data

### 一些示例

llama_factory 格式数据生产

In [ ]:
# sft 格式数据生成
df1 = pd.read_csv("train_data/video_script_data_gs_r1r2_pos_46w.csv").head(459000)
df1["lm_data"] = df1.apply(lambda x: gen_share_gpt_format_data(x["original_script"], x["final_script"]), axis=1)
print(df1.shape)
data_list = df1["lm_data"].to_list()
with open("xg_train_v2_46w_part1.json", "w") as f:
    json.dump(data_list[:200000], f, ensure_ascii=False, indent=4)


# dpo 格式数据生成
df1 = pd.read_csv("train_data/vs_data_0105_dpo_mid_9w.csv")
df1 = df1.dropna()
print(df1.shape)
df1["lm_data"] = df1.apply(lambda x: gen_dpo_format_data(x["original_script"], x["final_script"], x["rdc_res"]), axis=1)
with open("train_data/xg_train_dpo_v3_clean_2w.json", "w") as f:
    json.dump(df1["lm_data"].to_list(), f, ensure_ascii=False, indent=4)

# 未修改部分

In [ ]:

def __parse_log_data(text):
    data = json.loads(text)
    new_data = {
        "tid": data["raw"]["tid"],
        "question": data["base"]["question"],
        "answer": data["base"]["answer"],
        "raw_script": data["raw"]["script"],
        "script": data["res"]["script"],
        "err_info": data["res"]["err_info"],
    }
    return pd.Series(new_data)


def parse_log_data(df, colname="log"):
    df[["tid", "question", "answer", "raw_script", "script", "err_info"]] = df[colname].apply(__parse_log_data)
    return df

%pwd

'/Users/zyb/Documents/code'

In [43]:
def in_mmxg(row):
    if (row["pic"] == 1) and (row["jisuan"] == 0):
        if (row["has_gpt"] == 1) or (row["err_msg"] is not np.nan and row["err_msg"].strip('"') in {"题干是图片", "题干存在图片", "题干是图片"}):
            return 1
    return 0

df = pd.read_csv("mm/h2小数摸底.csv")
df = df.dropna(axis=1, how='all')
df["month"] = df["日期"].apply(lambda x : x.split("/")[1])
df["pic"] = df["是否含图"].apply(lambda x : 1 if x == "是" else 0)
df["jisuan"] = df.apply(lambda x : 1 if ((x["video_url"] is not np.nan) and (x["pic"] == 1)) else 0, axis=1)
df["has_gpt"] = df.apply(lambda x : 1 if (x["finally_script"] is not np.nan) and (x["pic"] == 1) else 0, axis=1)
df["mmxg_in"] = df.apply(in_mmxg, axis=1)
result = df.groupby('month').agg(
    tid_count=('tid', 'nunique'),  # 统计tid数量（排除重复）
    tid_total=('tid', 'count'),    # 统计tid出现次数（包含重复）
    pic_sum=('pic', 'sum'),         # 对pic列求和
    jisuan_sum=('jisuan', 'sum'),
    gpt_sum=('has_gpt', 'sum'),
    inmm_sum=('mmxg_in', 'sum'),
).reset_index()
result

/var/folders/ns/m5vxq5ds5f7gk9sblvwlhnf00000gn/T/ipykernel_51452/352234324.py:7: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("mm/h2小数摸底.csv")


,month,tid_count,tid_total,pic_sum,jisuan_sum,gpt_sum,inmm_sum
0,1,1970,1983,738,1,167,684
1,10,1942,1942,978,3,158,901
2,11,1941,1941,985,3,158,875
3,12,1961,1962,932,1,156,837
4,2,1958,1961,570,2,89,536
5,3,1981,1982,642,5,158,591
6,4,1984,1985,699,3,208,640
7,5,1969,1969,838,2,143,776
8,6,1961,1962,939,0,219,863
9,7,1957,1958,771,8,176,703


In [44]:
result.to_csv("mm/h2小数摸底汇总.csv", index=False)

In [49]:
df_in = df[df["mmxg_in"] == 1]
df1 = df_in[df_in["has_gpt"] == 1]
df2 = df_in[df_in["has_gpt"] == 0]
print(df1.shape)
print(df2.shape)
sampled_df1 = df1.groupby('month', group_keys=False).apply(lambda x: x.sample(n=50))
sampled_df2 = df2.groupby('month', group_keys=False).apply(lambda x: x.sample(n=50))
print(sampled_df1.shape)
print(sampled_df2.shape)

(1881, 20)
(6752, 20)
(600, 20)
(600, 20)


In [50]:
sampled_df1[["tid", "month", "finally_script"]].to_csv("mm/modi_gpt_sample600.csv", index=False)
sampled_df2[["tid", "month", "err_msg"]].to_csv("mm/modi_filtered_sample600.csv", index=False)

In [ ]:
df = df[df["是否含图"] == "是"]
# df = df[['日期', 'tid', '泛计算返回', 'err_msg', 'finally_script', 'ext' ]]
df = df[['日期', 'tid', 'err_msg', 'finally_script']]
df = df[df["finally_script"].isna()]
print(df.shape)
df.head(1)

In [215]:
def gen_original_script(text):
    try:
       data = eval(text)
    except:
        return ""

    try:
        new_data = {}
        cnt = 1
        for jt_data in data:
            for key in jt_data:
                new_data["镜头%d" % cnt ] = {
                    "内容类型": jt_data[key]["类型"],
                    "屏幕内容":jt_data[key].get("屏幕", ""),
                    "旁白内容":jt_data[key].get("旁白", ""),
                    "标注内容": "",
                }
                cnt += 1
    except:
        print(jt_data)
    return json.dumps(new_data, ensure_ascii=False)


def json_check(text):
    try:
       data = eval(text)
    except:
        return 0
    else:
        return 1


# df = pd.read_csv("mm_new/0915_review_raw.csv")
df = pd.read_excel("mm_new/output.xlsx")

df["json_check"] = df["result"].apply(json_check)
print(df[df["json_check"] == 0].shape)
df = df[df["json_check"] == 1]
print(df.shape)
df["original_script"] = df["result"].apply(gen_original_script)
df.to_csv("mm_new/mm_gpt5_0922_data_stage_1.csv", index=False)

(47, 10)
(151, 10)


In [179]:
# df = df[df["original_script"] != ""]
print(df.shape)
df = pd.concat([df, df1], ignore_index=True)
print(df.shape)
print(df.drop_duplicates(subset="tid").shape)

(117, 18)
(130, 18)
(130, 18)


In [47]:
df = pd.read_csv("aivideo_quality/data/raw/小数历史低分不合格脚本.csv")
df["script"] = df.apply(lambda x: x["original_script"] if x["model"] in {"改写", "泛计算"} else x["finally_script"], axis=1)
df = df[df["is_modified"] != 1][["tid", "teachertid", "script", "model"]]

df1 = pd.read_csv("aivideo_quality/data/raw/每周小数例行低分评估.csv")
df = pd.merge(df, df1, on="tid", how="left")
df.to_csv("aivideo_quality/data/bad_comment_util2508_dat_344.csv", index=False)

In [50]:
df = pd.read_csv("aivideo_quality/data/raw/小数H1周评脚本.csv")
df["script"] = df.apply(lambda x: x["original_script"] if x["model"] in {"改写", "泛计算"} else x["finally_script"], axis=1)
print(df.shape)
df = df[df["is_modified"] != 1][["tid", "teachertid", "script", "model"]]
print(df.shape)

df1 = pd.read_excel("aivideo_quality/data/raw/内审小数H1周评数据.xlsx")
df = pd.merge(df, df1, on="tid", how="left")
df.to_csv("aivideo_quality/data/week_review_25h1_data_799.csv", index=False)
df["视频等级"].value_counts()

(891, 7)
(799, 4)


视频等级
合格     357
优秀     356
不合格     86
Name: count, dtype: int64

In [117]:
def get_label(score):
    if score == 2:
        return "优秀"
    elif score == 0:
        return "不合格"
    else:
        return "合格"

df = pd.read_csv("rm4/data/raw/每周小数例行低分评估.csv")[['tid', '回流前错因', '回流后错因', '备注', '错误类型', '修改情况']]
df1 = pd.read_csv("rm4/data/raw/low_score_script.csv")

# df["label"] = df["label"].apply(get_label)
# df["info"] = df["info"].apply(lambda x: x.strip().strip("\n").strip("总结：") if type(x) == str else "")
df = pd.merge(df, df1, on="tid")
df["script"] = df.apply(lambda x: x["original_script"] if x["model"] in {"改写", "泛计算"} else x["finally_script"], axis=1)
df = df [['tid', '回流前错因', '回流后错因', '备注', '错误类型', '修改情况', 'model', 'status', 'script']]
df.to_csv("rm4/data/raw/low_score_script.csv", index=False)

In [125]:
def get_label(row):
    if row["错误类型"] == "没问题":
        return "合格"
    else:
        if row["model"] != "人工":
            return "不合格"
        else:
            if (row["status"] != 2.0) and (row["status"] != 4.0):
                return "不合格"
            else:
                return "-"


def get_err_info(row):
    e_type = row["错误类型"] if type(row["错误类型"]) == str else ""
    if type(row["回流后错因"]) == str:
        return e_type + "；" + row["回流后错因"]
    elif type(row["回流前错因"]) == str:
        return e_type + "；" + row["回流前错因"]
    else:
        return e_type
    

df = pd.read_csv("rm4/data/raw/low_score_script_fixed.csv")
# df["model"] = df["model"].fillna("人工")
# df["label"] = df.apply(get_label, axis=1)
df["info"] = df.apply(get_err_info, axis=1)
df.to_csv("rm4/data/raw/low_score_script_fixed.csv", index=False)

In [105]:
df.to_csv("rm4/data/review_data_y24_192.csv", index=False)

In [85]:
df = pd.read_csv("rm4/data/raw/m78_badcase_script.csv")
df1 = pd.read_csv("rm4/data/raw/m78_bad_comment_badcase.csv")
df= pd.merge(df, df1, on="tid")
df["script"] = df.apply(lambda x: x["original_script"] if x["model"] in {"改写", "泛计算"} else x["finally_script"], axis=1)
df.to_csv("rm4/data/m78_badcase.csv", index=False)

In [ ]:
df = pd.merge(df[["tid", "model", "script", "label"]], df1, on="tid")
print(df.drop_duplicates(subset="tid").shape)
df.to_csv("aivideo_quality/data/week_review_data_25h1_897.csv", index=False)

(897, 11)


In [39]:
df2 = pd.merge(df[["tid", "model", "script", "label"]], df1, on="tid")
df2 = df2.drop_duplicates(subset="tid")
df2.to_csv("aivideo_quality/data/bad_comment_data_end2508_396.csv", index=False)
# df2[df2.duplicated(subset="tid")]

In [199]:
df1 = pd.read_csv("mm_new/0915_gpt5_prompt_data.csv")
df2 = pd.read_excel("mm_new/小学数学-9&10月图题.xlsx")[["tid", "question_tol_url"]]
df = pd.merge(df1, df2, on="tid")
df.to_csv("mm_new/0915/0915_gpt5_prompt_data_new.csv", index=False)
# df = df2[~df2["tid"].isin(df1["tid"])]
# df["online"] = df.apply(lambda x: 1 if (x["RM3"] == 1) and (x["EQ"] == 1) else 0, axis=1)
# df[["tid", "online"]].to_csv("rm4/online_add_data.csv", index=False)


In [203]:
df1 = pd.read_excel("mm_new/0915/gpt5_res.xlsx")[["tid", "result"]]
df1["original_script"] = df1["result"].apply(gen_original_script)
df2 = pd.merge(df, df1, on="tid")
df2[df2["original_script"] == ""].shape

(34, 12)

In [138]:
def cal_matrix(df):
    total = df.shape[0]
    tp = df[(df["online"] == 1) & (df["label"] != "不合格")].shape[0]
    tn = df[(df["online"] == 0) & (df["label"] == "不合格")].shape[0]
    all_bad = df[df["label"] == "不合格"].shape[0]
    pick = df[df["online"] == 0].shape[0]
    c_rate = 0.0 if pick == 0 else tn / pick
    print((tp + tn )/ total, c_rate, tn / all_bad, sep="\t", end="\t")

cal_matrix(df)
cal_matrix(df[df["model"] == "人工"])
cal_matrix(df[df["model"] == "改写"])
cal_matrix(df[df["model"] == "泛计算"])

0.7833333333333333	0.6	0.2631578947368421	0.7625	0.5	0.3157894736842105	0.8875	0.75	0.2727272727272727	0.7	0.6666666666666666	0.2222222222222222	